# Tuflow Runner - Example Use

In [1]:
import modelrunner.tuflow as tfr

Set the Parameters of the run. Paths need to be entered as string literals. 

In [2]:
parameters: tfr.Parameters = tfr.Parameters(
    exec_path = r"C:/Program Files/TUFLOW",      #The path to the TUFLOW executable
    root = r"../tests/data/tuflow/one_tcf",      #The root of the model, location of the .tcf file.
    version = r"2020-10-AD",                     #The version of the engine
    engine = r"DP",                              #SP or DP
    flags = ['flag1', 'flage2', 'flag3'],        #Any number of flags
    async_runs = 2,                              #How many models to run at once
    stdout = r'./output/folder'                  #Where to put the model output (what TUFLOW prints to the console). No more poppy up windows!
)

Create a new tuflow runner with the parameter defined above. 

In [3]:
runner = tfr.Runner(parameters)

Create a Run. The run arguments need to match the tfc otherwise an error will occur when trying to stage the runs.

In [4]:
run = tfr.Run(s1="DES", e1="M720", e2="2100")

Stage that run in the runner.

In [5]:
runner.stage(run)

Loop through a bunch of scenarios and stage them in the runner. 

In [6]:
scenarios = ['EX', 'DES']
for s in scenarios:
    run = tfr.Run(
        s1=s,
        e1='M60',
        e2='CC'
    )
    runner.stage(run)

Display the runs that are staged.

In [7]:
runner.get_runs()

[{'s1': 'DES', 'e1': 'M720', 'e2': '2100'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Run the staged runs.

In [ ]:
runner.run()

Run a specific run number.

In [ ]:
runner.run('01')

Run multiple run numbers, this will alternate running each model. 

In [ ]:
runner.run('01', '02')

For sequetial running

In [ ]:
runner.run('01')
runner.run('02')

Create a new tuflow runner from other runners. The staged runs are inherited from other runners. 

In [10]:
runner_1 = tfr.Runner(parameters)
runner_2 = tfr.Runner(parameters)
runner_3 = tfr.Runner(parameters)

runner_1.stage(tfr.Run(s1='EX', e1='M120', e2='CC'))
runner_2.stage(tfr.Run(s1='DES', e1='M120', e2='CC'))
runner_3.stage(tfr.Run(s1='DES', e1='M45', e2='NC'))

runner_all = tfr.Runner(parameters, runner_1, runner_2, runner_3)

runner_all.get_runs()

[{'s1': 'EX', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M45', 'e2': 'NC'}]

Duplicates are not added.

In [11]:
runner_all.stage(tfr.Run(s1='EX', e1='M120', e2='CC'))
runner_all.stage(tfr.Run(s1='DES', e1='M120', e2='CC'))
runner_all.stage(tfr.Run(s1='DES', e1='M45', e2='NC'))

runner_all.get_runs()

[{'s1': 'EX', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M45', 'e2': 'NC'}]

Filter runs based on any arguments in a list. Lets get all the DES runs.

In [12]:
des_runs = runner.get_runs('DES')
des_runs

[{'s1': 'DES', 'e1': 'M720', 'e2': '2100'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Now get all DES and EX runs.

In [13]:
des_ex_runs = runner.get_runs('DES', 'EX')
des_ex_runs

[{'s1': 'DES', 'e1': 'M720', 'e2': '2100'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Filter runs that match ALL listed arguments. No runs satisfy both DES and EX scenarios.

In [14]:
no_runs = runner.get_runs('DES', "EX", any=False)
no_runs

[]

Remove runs from a runner. Lets remove the DES runs from the runner, we retrieved above.

In [15]:
runner_remove = tfr.Runner(parameters, runner)
runner_remove.remove_runs(des_runs)
runner_remove.get_runs()

[{'s1': 'EX', 'e1': 'M60', 'e2': 'CC'}]

Stage runs that are in a list.

In [16]:
runner_subset = tfr.Runner(parameters)
des_runs = runner.get_runs('DES')
runner_subset.stage(des_runs) 

runner_subset.get_runs()

[{'s1': 'DES', 'e1': 'M720', 'e2': '2100'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Get a run from a runner by index. Get the first run added to the runner.

In [18]:
run_indexed = runner[0]
run_indexed

{'s1': 'DES', 'e1': 'M720', 'e2': '2100'}

Or by slice. Get the first two runs added to the runner.

In [19]:
runner_sliced = runner[:2]
runner_sliced

[{'s1': 'DES', 'e1': 'M720', 'e2': '2100'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'}]

Make toggle testing straight forward, by setting parameter on the fly. The parameter must have been intially set when the Parameters object was created.

In [ ]:
TEST = True

if TEST:
    parameters.flags = ['test']
    runner.run()
else:
    parameters.flags = ['batch']
    runner.run()

Change versions on the fly.

In [ ]:
parameters.version = '2020-10-AF'
runner.run()